> ## US Drought & Meteorological Data Starter Notebook
This notebook will walk you trough loading the data and create a Dummy Classifier, showing a range of F1 scores that correspond to random predictions if given theclass priors.

## Loading & Visualizing the Data
In this section, we load the training and validation data into numpy arrays and visualize the drought classes and meteorological attributes.

We load the json files for training, validation and testing into the ``files`` dictionary.

In [1]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from datetime import datetime
from functools import lru_cache
sns.set_style('white')

files = {}

for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        if 'train' in filename:
            files['train'] = os.path.join(dirname, filename)
        if 'valid' in filename:
            files['valid'] = os.path.join(dirname, filename)
        if 'test' in filename:
            files['test'] = os.path.join(dirname, filename)

The following classes exist, ranging from no drought (``None``), to extreme drought (``D4``).
This could be treated as a regression, ordinal or classification problem, but for now we will treat it as 5 distinct classes.

In [2]:
class2id = {
    'None': 0,
    'D0': 1,
    'D1': 2,
    'D2': 3,
    'D3': 4,
    'D4': 5,
}
id2class = {v: k for k, v in class2id.items()}

Now we'll define a helper method to load the datasets. This just walks through the json and discards the few samples that are corrupted.

In [3]:
dfs = {
    k: pd.read_csv(files[k]).set_index(['fips', 'date'])
    for k in files.keys()
}

In [6]:
import numpy as np
from scipy.interpolate import interp1d

def interpolate_nans(padata, pkind='linear'):
    """
    see: https://stackoverflow.com/a/53050216/2167159
    """
    aindexes = np.arange(padata.shape[0])
    agood_indexes, = np.where(np.isfinite(padata))
    f = interp1d(agood_indexes
               , padata[agood_indexes]
               , bounds_error=False
               , copy=False
               , fill_value="extrapolate"
               , kind=pkind)
    return f(aindexes)

In [53]:
@lru_cache(maxsize=1000)
def date_encode(date):
    if isinstance(date, str):
        date = datetime.strptime(date, '%Y-%m-%d')
    return (
        np.sin(2*np.pi*date.timetuple().tm_yday/366),
        np.cos(2*np.pi*date.timetuple().tm_yday/366)
    )

In [54]:
# load one of 'train', 'valid' or 'test'
def loadXY(
    df,
    shuffle=True,
    random_state=42,
    window_size=180,
    target_size=12,
    fuse_past=False,
    return_fips=False,
    encode_season=False,
):
    soil_df = pd.read_csv('soil_data.csv')
    time_data_cols = sorted([c for c in df.columns if c not in ['fips','date','score']])
    static_data_cols = sorted([c for c in soil_df.columns if c not in ['soil','lat','lon']])
    count = 0
    score_df = df.dropna(subset=['score'])
    X_static = np.empty((len(df)//window_size, len(static_data_cols)))
    X_fips_date = []
    add_dim = 0
    if fuse_past:
        add_dim += 1
    if encode_season:
        add_dim += 2
    X_time = np.empty((len(df)//window_size, window_size, len(time_data_cols)+add_dim))
    y_past = np.empty((len(df)//window_size, window_size))
    y_target = np.empty((len(df)//window_size, target_size))
    if random_state is not None:
        np.random.seed(random_state)
    for fips in tqdm(score_df.index.get_level_values(0).unique()):
        if random_state is not None:
            start_i = np.random.randint(1, window_size)
        else:
            start_i = 1
        fips_df = df[(df.index.get_level_values(0)==fips)]
        X = fips_df[time_data_cols].values
        y = fips_df['score'].values
        X_s = soil_df[soil_df['fips']==fips][static_data_cols].values[0]
        for i in range(start_i, len(y)-(window_size+target_size*7), window_size):
            X_fips_date.append((fips,fips_df.index[i:i+window_size][-1]))
            X_time[count,:,:len(time_data_cols)] = X[i:i+window_size]
            if not fuse_past:
                y_past[count] = interpolate_nans(y[i:i+window_size])
            else:
                X_time[count,:,len(time_data_cols)] = interpolate_nans(y[i:i+window_size])
            if encode_season:
                enc_dates = [date_encode(d) for f, d in fips_df.index[i:i+window_size].values]
                d_sin, d_cos = [s for s, c in enc_dates], [c for s, c in enc_dates]
                X_time[count,:,len(time_data_cols)+(add_dim-2)] = d_sin
                X_time[count,:,len(time_data_cols)+(add_dim-2)+1] = d_cos
            temp_y = y[i+window_size:i+window_size+target_size*7]
            y_target[count] = np.array(temp_y[~np.isnan(temp_y)][:target_size])
            X_static[count] = X_s
            count += 1
    results = [X_static[:count], X_time[:count], y_target[:count]]
    if not fuse_past:
        results.append(y_past[:count])
    if return_fips:
        results.append(X_fips_date)
    return results

In [55]:
scaler_dict = {}
scaler_dict_static = {}
scaler_dict_past = {}

In [56]:
from sklearn.preprocessing import RobustScaler

def normalize(X_static, X_time, y_past=None, fit=False):
    for index in tqdm(range(X_time.shape[-1])):
        if fit:
            scaler_dict[index] = RobustScaler().fit(
                X_time[:,:,index].reshape(-1, 1)
            )
        X_time[:,:,index] = scaler_dict[index].transform(X_time[:,:,index].reshape(-1, 1)).reshape(-1, X_time.shape[-2])
    for index in tqdm(range(X_static.shape[-1])):
        if fit:
            scaler_dict_static[index] = RobustScaler().fit(
                X_static[:,index].reshape(-1, 1)
            )
        X_static[:,index] = scaler_dict_static[index].transform(X_static[:,index].reshape(-1, 1)).reshape(1, -1)
    index = 0
    if y_past is not None:
        if fit:
            scaler_dict_past[index] = RobustScaler().fit(
                y_past.reshape(-1, 1)
            )
        y_past[:,:] = scaler_dict_past[index].transform(y_past.reshape(-1, 1)).reshape(-1, y_past.shape[-1])
        return X_static, X_time, y_past
    return X_static, X_time

In [57]:
X_static_train, X_time_train, y_target_train = loadXY(dfs['train'], fuse_past=True, encode_season=True)

  0%|          | 0/3108 [00:00<?, ?it/s]

In [59]:
X_static_valid, X_time_valid, y_target_valid, valid_fips = loadXY(dfs['valid'], random_state=None, fuse_past=True, return_fips=True, encode_season=True)

  0%|          | 0/3108 [00:00<?, ?it/s]

In [60]:
X_static_train, X_time_train = normalize(X_static_train, X_time_train, fit=True)

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [61]:
X_static_valid, X_time_valid = normalize(X_static_valid, X_time_valid)

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [62]:
# hyper parameters
batch_size = 128
lr = 7e-5
output_size = 1
hidden_dim = 512
dropout = 0.1
n_layers = 2
epochs = 10
clip = 5

In [63]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.tensor(X_time_train), torch.tensor(y_target_train[:,0]))
train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size, drop_last=True)
valid_data = TensorDataset(torch.tensor(X_time_valid), torch.tensor(y_target_valid[:,0]))
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size, drop_last=True)

In [65]:
import torch
from torch import nn

class DroughtNetLSTM(nn.Module):
    def __init__(self, output_size, num_input_features, hidden_dim, n_layers, drop_prob=0.2, add_static=False):
        super(DroughtNetLSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(num_input_features, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden, static=None):
        batch_size = x.size(0)
        x = x.cuda().to(dtype=torch.float32)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.fc2(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        )
        return hidden

In [66]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print('using GPU')
else:
    device = torch.device("cpu")
    print('using CPU')


model = DroughtNetLSTM(output_size, X_time_train.shape[-1], hidden_dim, n_layers, dropout)
model.to(device)

using GPU


DroughtNetLSTM(
  (lstm): LSTM(21, 512, num_layers=2, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)

In [67]:
loss_function = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(train_loader), epochs=epochs)

In [ ]:
from sklearn.metrics import f1_score

counter = 0
valid_loss_min = np.Inf
torch.manual_seed(42)
np.random.seed(42)

for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for k, (inputs, labels) in tqdm(enumerate(train_loader), desc=f'epoch {i+1}/{epochs}', total=len(train_loader)):
        model.train()
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = loss_function(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        scheduler.step()
        
        if k == len(train_loader) - 1 or k == (len(train_loader) - 1) // 2:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            labels = []
            preds = []
            for inp, lab in valid_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = loss_function(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                for l in lab:
                    labels.append(int(l))
                for p in out.round():
                    if p > 5:
                        p = 5
                    if p < 0:
                        p = 0
                    preds.append(int(p))
            
            # log data
            log_dict = {
                'loss': float(loss),
                'epoch': counter/len(train_loader),
                'step': counter,
                'lr': scheduler.get_last_lr()[0]
            }
            log_dict['validation_loss'] = np.mean(val_losses)
            log_dict[f'macro_f1'] = f1_score(labels, preds, average='macro')
            log_dict[f'micro_f1'] = f1_score(labels, preds, average='micro')
            for j, f1 in enumerate(f1_score(labels, preds, average=None)):
                log_dict[f'{id2class[j]}_f1'] = f1
            print(log_dict)
            
            model.train()
            
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
                best_log_dict = log_dict

epoch 1/10:   0%|          | 0/802 [00:00<?, ?it/s]

{'loss': 0.8793061971664429, 'epoch': 0.5, 'step': 401, 'lr': 7.3052046939180236e-06, 'validation_loss': 0.557697421974606, 'macro_f1': 0.230593548536648, 'micro_f1': 0.7230902777777778, 'None_f1': 0.8698350481789972, 'D0_f1': 0.5137262430408908, 'D1_f1': 0.0, 'D2_f1': 0.0, 'D3_f1': 0.0, 'D4_f1': 0.0}
Validation loss decreased (inf --> 0.557697).  Saving model ...
{'loss': 0.06499575823545456, 'epoch': 1.0, 'step': 802, 'lr': 1.9612671791499418e-05, 'validation_loss': 0.10352242758704557, 'macro_f1': 0.5870767269519013, 'micro_f1': 0.8684895833333334, 'None_f1': 0.9469095438268722, 'D0_f1': 0.7129551227773073, 'D1_f1': 0.6707734428473648, 'D2_f1': 0.5752808988764045, 'D3_f1': 0.6165413533834586, 'D4_f1': 0.0}
Validation loss decreased (0.557697 --> 0.103522).  Saving model ...


epoch 2/10:   0%|          | 0/802 [00:00<?, ?it/s]

{'loss': 0.08812756836414337, 'epoch': 1.5, 'step': 1203, 'lr': 3.642194542737123e-05, 'validation_loss': 0.0818952369865858, 'macro_f1': 0.720884762279371, 'micro_f1': 0.8619791666666666, 'None_f1': 0.9419446192573946, 'D0_f1': 0.7008071249652101, 'D1_f1': 0.6821705426356589, 'D2_f1': 0.5688073394495413, 'D3_f1': 0.631578947368421, 'D4_f1': 0.8}
Validation loss decreased (0.103522 --> 0.081895).  Saving model ...
{'loss': 0.034090444445610046, 'epoch': 2.0, 'step': 1604, 'lr': 5.32253340250048e-05, 'validation_loss': 0.06694786864358725, 'macro_f1': 0.6083025623710278, 'micro_f1': 0.8509114583333334, 'None_f1': 0.9370994540707335, 'D0_f1': 0.6832541235672351, 'D1_f1': 0.6753894080996885, 'D2_f1': 0.5286343612334802, 'D3_f1': 0.4776119402985075, 'D4_f1': 0.34782608695652173}
Validation loss decreased (0.081895 --> 0.066948).  Saving model ...


epoch 3/10:   0%|          | 0/802 [00:00<?, ?it/s]

...

{'loss': 0.02932736463844776, 'epoch': 4.0, 'step': 3208, 'lr': 6.652542129575786e-05, 'validation_loss': 0.06446024133280541, 'macro_f1': 0.5973752880593434, 'micro_f1': 0.8549262152777778, 'None_f1': 0.9434703986162434, 'D0_f1': 0.6925287356321839, 'D1_f1': 0.6575171553337492, 'D2_f1': 0.47639484978540775, 'D3_f1': 0.4507042253521127, 'D4_f1': 0.3636363636363636}
Validation loss decreased (0.065140 --> 0.064460).  Saving model ...


epoch 5/10:   0%|          | 0/802 [00:00<?, ?it/s]

{'loss': 0.07341979444026947, 'epoch': 4.5, 'step': 3609, 'lr': 6.235191653044496e-05, 'validation_loss': 0.0638828135561198, 'macro_f1': 0.654791209019074, 'micro_f1': 0.8531901041666666, 'None_f1': 0.9396068524512513, 'D0_f1': 0.6827332010206975, 'D1_f1': 0.6670769230769231, 'D2_f1': 0.5324675324675324, 'D3_f1': 0.5735294117647058, 'D4_f1': 0.5333333333333333}
Validation loss decreased (0.064460 --> 0.063883).  Saving model ...
{'loss': 0.027604930102825165, 'epoch': 5.0, 'step': 4010, 'lr': 5.680687947882316e-05, 'validation_loss': 0.06433719060603632, 'macro_f1': 0.6186951743337877, 'micro_f1': 0.8564453125, 'None_f1': 0.9428706326723325, 'D0_f1': 0.6937142857142857, 'D1_f1': 0.6633354153653965, 'D2_f1': 0.5085470085470085, 'D3_f1': 0.5037037037037037, 'D4_f1': 0.4}


epoch 6/10:   0%|          | 0/802 [00:00<?, ?it/s]

{'loss': 0.07400666177272797, 'epoch': 5.5, 'step': 4411, 'lr': 5.016836145271907e-05, 'validation_loss': 0.06517152549349703, 'macro_f1': 0.6591314639464531, 'micro_f1': 0.8562282986111112, 'None_f1': 0.9406104582380314, 'D0_f1': 0.6878547105561862, 'D1_f1': 0.6765067650676507, 'D2_f1': 0.5450549450549451, 'D3_f1': 0.5714285714285715, 'D4_f1': 0.5333333333333333}


In [ ]:
model.eval()
val_h = tuple([each.data for each in model.init_hidden(1)])

In [31]:
def predict(x):
    out, _ = model(torch.tensor(x), val_h)
    return out

In [33]:
dict_map = {
    'y_pred': [],
    'y_pred_rounded': [],
    'fips': [],
    'date': [],
    'y_true': [],
}
for x, fips_date, y in tqdm(zip(X_time_valid, valid_fips, y_target_valid), total=len(X_time_valid)):
    pred = predict(torch.tensor([x])).clone().detach()[0].float()
    if fips_date[1] not in dict_map['fips']:
        dict_map['y_pred'].append(float(pred))
        dict_map['y_pred_rounded'].append(int(pred.round()))
        dict_map['fips'].append(fips_date[1][0])
        dict_map['date'].append(fips_date[1][1])
        dict_map['y_true'].append(y[0])

  0%|          | 0/9324 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


In [34]:
df = pd.DataFrame(dict_map)

In [35]:
df.to_csv('drougths_2018.csv')

In [38]:
f1_score(df['y_true'].apply(round), df['y_pred_rounded'], average=None)

array([0.10358689, 0.368     , 0.09124933, 0.13235294, 0.        ,
       0.        ])

In [42]:
dates = df['date'].unique()

In [47]:
df0 = df[df['date']==dates[0]]
df1 = df[df['date']==dates[1]]
df2 = df[df['date']==dates[2]]

In [48]:
f1_score(df0['y_true'].apply(round), df0['y_pred_rounded'], average='macro')

0.016426703215121925

In [49]:
f1_score(df1['y_true'].apply(round), df1['y_pred_rounded'], average='macro')

0.23451370512081243

In [53]:
f1_score(df2['y_true'].apply(round), df2['y_pred_rounded'], average='macro')

0.04124057804023237

In [56]:
from sklearn.metrics import mean_absolute_error

In [57]:
mean_absolute_error(df2['y_true'], df2['y_pred'])

1.9268948446595375

In [58]:
mean_absolute_error(df1['y_true'], df1['y_pred'])

0.6846835148191973

In [59]:
mean_absolute_error(df0['y_true'], df0['y_pred'])

2.2092584471222056